# **<center>SETI Break through<center>**

First of all we have to deal with the required packages.  

In [172]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from itertools import chain

Now we will read our labels and get the data it self.

In [205]:
nrows=2500
train_dir = '/kaggle/input/seti-breakthrough-listen/train'
test_dir = '/kaggle/input/seti-breakthrough-listen/test'
train_df = pd.read_csv('/kaggle/input/seti-breakthrough-listen/train_labels.csv',nrows=nrows)
test_df = pd.read_csv('/kaggle/input/seti-breakthrough-listen/sample_submission.csv',nrows=nrows)

In [174]:
def return_filepath(name, folder=train_dir):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

In [206]:
train_df['image_path'] = train_df['id'].apply(lambda x: return_filepath(x))
test_df['image_path'] = test_df['id'].apply(lambda x: return_filepath(x, folder=test_dir))
train_df.head()

,id,target,image_path
0,0024012d1431fbc,1,./train\0\0024012d1431fbc.npy
1,1004c26b9a3f877,1,./train\1\1004c26b9a3f877.npy
2,0000799a2b2c42d,0,./train\0\0000799a2b2c42d.npy
3,0012ef703e41ef0,0,./train\0\0012ef703e41ef0.npy
4,00042890562ff68,0,./train\0\00042890562ff68.npy


In [207]:
train_data = []
for train_image_path in train_df['image_path']:
    image = (np.load(train_image_path))
    
    # 1
  #  on = np.vstack((image[0],image[2],image[4]))
  #  off = np.vstack((image[1], image[3], image[5]))
  #  image_merged = np.concatenate((on, off), axis=1).transpose((1, 0))


    # 2
    image_merged = np.vstack(image).transpose((1, 0))

    # 3
  #  np.vstack((image[0],image[1],image[2],image[3],image[4],image[5]))

    # 4
  #  on = np.vstack((image[0],image[2],image[4]))
  #  off = np.vstack((image[1], image[3], image[5]))
  #  image_merged = np.concatenate((on, off), axis=1))

    train_data.append(image_merged.flatten())

In [177]:
models = []
models.append(('LR', SGDClassifier(loss='log_loss')))
models.append(('PT', SGDClassifier(loss='perceptron')))
models.append(('SVM', SGDClassifier(loss='hinge')))

In [ ]:
k_folds = 10
kf = StratifiedKFold(n_splits=k_folds)

model = SGDClassifier()

batch_size = nrows

if nrows >= 5000:
    batch_size = 1000

for name, model in models:
    fold_scores = []
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for train_index, test_index in kf.split(train_data, train_df['target']):

        for i in train_index:       
            X_train.append(train_data[i])
            y_train.append(train_df['target'][i])

        for i in test_index:       
            X_test.append(train_data[i])
            y_test.append(train_df['target'][i])

        for i in range(0, len(X_train), batch_size):
            X_batch, y_batch = X_train[i:i+batch_size], y_train[i:i+batch_size]
            model.partial_fit(X_batch, y_batch, classes=np.unique(train_df['target']))

        # Evaluate the model
        fold_score = model.score(X_test, y_test)
        fold_scores.append(fold_score)

    print("Cross-validation scores for each fold:", fold_scores)
